# Lago Workshop!
Introduction to Lago CLI and SDK.

Lab 0
=====

**Goal: Get your machine setup for running the lab**

Clone the workshop repository from GitHub:
```bash

$ git clone https://github.com/lago-project/lago-workshop.git
```

Switch to the workshop virtual enviornment by running:

```bash

$ workon lago-workshop
```

If you are not working with virtualenv, install the ``requirements.txt`` in the root directory of the cloned workshop by running:
```bash
$ pip -I -r requirements.txt
```

Lab A
=====

** Goal: Get to know Lago Command Line **

Ensure Lago is working on your laptop by running:
```bash
$ lago --version
```

Change to your to the lago-workshop first lab example:

```bash
$ cd ~/lago-workshop/intro
```

Explore the init file by running:
```bash
$ cat init-simple.yaml
```

Initalize the virtual enviornment:
```bash
$ lago init
```

Start the enviornment:
```bash
$ lago start
```

When the enviornment is up, run ``lago status``, notice that the VM ``vm-01`` status should be "running"

Now execute a command on the VM with SSH, we will check the VM's uptime:
```bash
$ lago shell test-vm01 -c 'uptime'
```

Exercise
*********
Interactive shell can be done by running ``lago shell test-vm01``, this is parallel to to "normal" ssh you would to a remote machine. Log into the machine and extract which distribution it is, normally this information is stored in a file with the glob  matching ``/etc/*release*``.





<details>
  <summary>Click here for for answer</summary>
  Thats right, it is CentOS 7.3!, this information can be extracted by SSH'ing into the VM and running cat /etc/*release*.
</details>



Finally, lets destroy the enviornment, run inside the same directory:
```bash
$ lago destroy
```




Lab B
=====
**Goal: Working with Lago SDK!**

Goal: Initalize the Lago enviornment, directly from Python.

Change to the workshop intro directory:
```bash
$ cd ~/lago-workshop/intro
```

Start an interactive IPython session by running:
```bash
$ ipython
```

Inside the IPython console, first import the dependencies:

In [1]:
import logging
from lago import sdk

Now create the same environment by calling ``sdk.init``, also we'll setup logging for debugging at ``/tmp/lago.log``:

In [3]:
env = sdk.init(config='init-simple.yaml',
              workdir='/tmp/lago_sdk_simple_example',
              loglevel=logging.DEBUG,
              log_fname='/tmp/lago.log')


Start the environment:

In [4]:
env.start()

Check which VMs are available by using the ``get_vms()`` method:

In [6]:
vms = env.get_vms()
print vms

{'vm-01': <lago.vm.DefaultVM object at 0x7f793158a2d0>}


In [7]:
vm = vms['vm-01']

Now we'll print some metadata available at the virtual machine object, namely the VM's IP and Linux distribution:

In [8]:
distro = vm.distro()
print distro
ip = vm.ip()
print ip

el7
192.168.200.2


Now lets run an automated SSH command to the VM:

In [9]:
res = vm.ssh(['hostname', '-f'])

The returned object of type ``CommandStatus`` includes 3 variabiles: ``out``, ``err`` which contain the SSH command standard output, standard output and finally the ``code`` which is the command's exit code on the VM.

In [10]:
res

CommandStatus(code=0, out='vm-01.lago.local\n', err='')

*Exercise* : Try running a command that will not work, and check what is the error code. 

Another important functionality of Lago, is to collect existing files from the virtual machine. This is the function signature:
```python
    def collect_artifacts(self, output_dir, ignore_nopath):
```
The ``output_dir`` parameter, is where you would like to collect those files. The files to collect are defined in the LagoInitFile, if not defined it will default to ``/var/log``.
*Note that if the output_dir already exists it will try to rotate it, so never do it on your home directory. This is designed that way so you could collect results of several different runs to the same directory*

First open another terminal window, with that in hand run:


In [11]:
env.collect_artifacts(output_dir='/tmp/myluckyday', ignore_nopath=True)

Now on your other terminal window, run:
```bash
$ tree /tmp/myluckyday
```
You should see a nice structutre of directories composed of the VM name, and the path which was collected.
Now run the same command again, and check out the ``/tmp/myluckday`` directory in your terminal, you should see that logs were collected twice. 

In [12]:
env.collect_artifacts(output_dir='/tmp/myluckyday', ignore_nopath=True)

Now exit from your IPython session and login again, and we will see how you can reload the existing environment: 

In [13]:
import logging
from lago import sdk

In [14]:
new_env = sdk.load_env(workdir='/tmp/lago_sdk_simple_example')

In [15]:
new_env.destroy()